In [ ]:
. ../../../../scripts/nbs_header.ps1
. ../../../../scripts/core.ps1

In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(GetExecutableSuffix) --execute-command "pwsh -c `"../../../../scripts/invoke-dib.ps1 test.dib`"" } | Invoke-Block -Retries 5

00:00:00 #1 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:00 #2 [Debug] executeAsync / options: { Command =
   "dotnet "/home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release/Spiral.dll" --port 13805 --default-int i32 --default-float f64"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:main@434-47> }
00:00:00 #4 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:00 #3 [Verbose] > pwd: /home/runner/work/polyglot/polyglot/apps/spiral/temp/test
00:00:00 #5 [Verbose] > dll_path: /home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release
00:00:00 #6 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:00 #7 [Verbose] testPortOpen / ex: System.AggregateException: 

In [ ]:
{ . ../../../../apps/parser/dist/DibParser$(GetExecutableSuffix) test.dib spi } | Invoke-Block

00:00:00 #1 [Debug] writeDibCode / output: Spi / path: test.dib
00:00:00 #2 [Debug] parseDibCode / output: Spi / file: test.dib


In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(GetExecutableSuffix) --build-file test.spi test.fsx --timeout 10000 } | Invoke-Block

00:00:00 #1 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:00 #2 [Debug] executeAsync / options: { Command =
   "dotnet "/home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release/Spiral.dll" --port 13805 --default-int i32 --default-float f64"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:main@434-47> }
00:00:00 #3 [Verbose] > pwd: /home/runner/work/polyglot/polyglot/apps/spiral/temp/test
00:00:00 #4 [Verbose] > dll_path: /home/runner/work/polyglot/polyglot/deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release
00:00:00 #5 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (Connection refused)
00:00:00 #6 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:00 #7 [Verbose] testPortOpen / ex: System.AggregateException: 

In [ ]:
dotnet fable --optimize --lang rs --extension .rs

Fable 4.14.0: F# to Rust compiler (status: alpha)

Thanks to the contributor! @SCullman
Stand with Ukraine! https://standwithukraine.com.ua/

Parsing test.fsproj...
.> dotnet restore test.fable-temp.csproj -p:FABLE_COMPILER=true -p:FABLE_COMPILER_4=true -p:FABLE_COMPILER_RUST=true
  Determining projects to restore...
  Paket version 8.1.0-alpha001+8e77fe66ac27ddecf519e0df5a4d3327ca9250e2
  The last full restore is still up to date. Nothing left to do.
  Total time taken: 0 milliseconds
  Paket version 8.1.0-alpha001+8e77fe66ac27ddecf519e0df5a4d3327ca9250e2
  Restoring /home/runner/work/polyglot/polyglot/apps/spiral/temp/test/test.fable-temp.csproj
  Starting restore process.
  Total time taken: 0 milliseconds
  Restored /home/runner/work/polyglot/polyglot/apps/spiral/temp/test/test.fable-temp.csproj (in 240 ms).
.> dotnet restore /home/runner/work/polyglot/polyglot/apps/spiral/temp/test/test.fsproj
  Determining projects to restore...
  Restored /home/runner/work/polyglot/polyglot/apps

In [ ]:
(Get-Content test.rs) `
    -replace [regex]::Escape("),);"), "));" `
| Set-Content test.rs

In [ ]:
cargo fmt --

In [ ]:
cargo build --release

   Compiling linux-raw-sys v0.4.13
   Compiling syn v2.0.57
   Compiling num-traits v0.2.18
   Compiling lazy_static v1.4.0
   Compiling rand_chacha v0.3.1
   Compiling rustix v0.38.32
   Compiling libm v0.2.8
   Compiling wait-timeout v0.2.0
   Compiling quick-error v1.2.3
   Compiling bit-vec v0.6.3
   Compiling bit-set v0.5.3
   Compiling rand v0.8.5
   Compiling tempfile v3.10.1
   Compiling sharded-slab v0.1.7
   Compiling rusty-fork v0.3.0
   Compiling rand_xorshift v0.3.0
   Compiling memchr v2.7.2
   Compiling unarray v0.1.4
   Compiling proptest v1.4.0
   Compiling tracing-subscriber v0.3.18
   Compiling nom v7.1.3
   Compiling fable_library_rust v0.1.0 (/home/runner/work/polyglot/polyglot/lib/rust/fable/fable_modules/fable-library-rust)
   Compiling tracing-attributes v0.1.27
   Compiling thiserror-impl v1.0.58
   Compiling thiserror v1.0.58
   Compiling tracing v0.1.40
   Compiling spiral_temp_test v0.0.1 (/home/runner/work/polyglot/polyglot/apps/spiral/temp/test)
  --> apps

In [ ]:
$env:RUST_LOG="info"
{ cargo test --release } | Invoke-Block

   Compiling once_cell v1.19.0
   Compiling bitflags v2.5.0
   Compiling linux-raw-sys v0.4.13
   Compiling fastrand v2.0.2
   Compiling tracing-core v0.1.32
   Compiling wait-timeout v0.2.0
   Compiling bit-vec v0.6.3
   Compiling quick-error v1.2.3
   Compiling log v0.4.21
   Compiling rustix v0.38.32
   Compiling fnv v1.0.7
   Compiling overload v0.1.1
   Compiling tracing-log v0.2.0
   Compiling nu-ansi-term v0.46.0
   Compiling bit-set v0.5.3
   Compiling num-traits v0.2.18
   Compiling rand v0.8.5
   Compiling thread_local v1.1.8
   Compiling sharded-slab v0.1.7
   Compiling rand_xorshift v0.3.0
   Compiling regex-syntax v0.8.3
   Compiling minimal-lexical v0.2.1
   Compiling unarray v0.1.4
   Compiling memchr v2.7.2
   Compiling pin-project-lite v0.2.14
   Compiling tracing v0.1.40
   Compiling tempfile v3.10.1
   Compiling rusty-fork v0.3.0
   Compiling nom v7.1.3
   Compiling tracing-subscriber v0.3.18
   Compiling thiserror v1.0.58
   Compiling fable_library_rust v0.1.0 (/hom

In [ ]:
{ . $ScriptDir/../../../../target/release/spiral_temp_test$(GetExecutableSuffix) } | Invoke-Block

app=test
